<a href="https://colab.research.google.com/github/Denise-Pro/analise-homicidios-rj-e-sp-com-spark-e-powerBI/blob/master/case_unificando%20e%20analisando%20com%20spark%20no%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')


# iniciar uma sessão local e importar dados do Airbnb
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()


In [4]:
from pyspark.sql import *

In [8]:
# download do http para arquivo local
!wget --quiet --show-progress https://noverde-data-engineering-test.s3.amazonaws.com/loans_sample.csv

# carregando dados do arquivo Csv
df_spark_csv = sc.read.csv("./loans_sample.csv", inferSchema=True, header=True)

# ver algumas informações sobre os tipos de dados de cada coluna
df_spark_csv.printSchema()

loans_sample.csv.1  100%[===================>]  44.36K  --.-KB/s    in 0.06s   
root
 |-- loan_id: integer (nullable = true)
 |-- period: integer (nullable = true)
 |-- accepted_at: timestamp (nullable = true)
 |-- payday: integer (nullable = true)
 |-- interest_rate: double (nullable = true)



In [13]:
# download do http para arquivo local
!wget --quiet --show-progress https://noverde-data-engineering-test.s3.amazonaws.com/installments_sample.json

# carregando dados do arquivo Csv
df_spark_json = sc.read.json("./installments_sample.json")

# ver algumas informações sobre os tipos de dados de cada coluna
display(df_spark_json)

installments_sample 100%[===================>] 983.37K  3.10MB/s    in 0.3s    


DataFrame[data: array<struct<due_date:string,installment_id:bigint,installment_value:double,loan_id:bigint,number:bigint>>, schema: struct<fields:array<struct<name:string,type:string>>,pandas_version:string,primaryKey:array<string>>]

In [18]:
dataset = sc.read.format("json").option("multiLine",True).load("./installments_sample.json")
display(dataset)

DataFrame[data: array<struct<due_date:string,installment_id:bigint,installment_value:double,loan_id:bigint,number:bigint>>, schema: struct<fields:array<struct<name:string,type:string>>,pandas_version:string,primaryKey:array<string>>]

In [20]:
# download do http para arquivo local
!wget --quiet --show-progress https://noverde-data-engineering-test.s3.amazonaws.com/payments_sample.parquet
data = sc.read.format("parquet").load("./payments_sample.parquet")
data.printSchema()

payments_sample.par 100%[===================>] 253.90K  1.02MB/s    in 0.2s    
root
 |-- loan_id: long (nullable = true)
 |-- installment_id: long (nullable = true)
 |-- payment_date: date (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- payment_id: string (nullable = true)
 |-- paid_amount: double (nullable = true)

